In [2]:
from docx import Document
from pandas import DataFrame, concat
import pandas as pd
import re
import os

In [3]:
def read_docx(file_path):
    doc = Document(file_path)
    return doc

In [4]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return "\n".join(full_text)


In [6]:
def df_find_cases(file_path):
    words_for_table = []  # Use a list to store repeated entries
    sujeito_list = [] # Initialize sujeito_list outside the loop
    curso_list = []  # Initialize curso_list outside the loop
    sexo_list = [] # Initialize sexo_list outside the loop
    ano_list = [] # Initialize ano_list outside the loop
    cidade_list = [] # Initialize cidade_list outside the loop

    # Iterate through all files in the specified directory
    for filename in os.listdir(directory_path):
        if filename.endswith(".docx"):
            file_path = os.path.join(directory_path, filename)
            text = read_docx(file_path)

            ### WORDS WITH PATTERN ###
            pattern = r'\S*\[[^\]]*(?:ndʊ|ndo|n∅dʊ|n∅do)[^\]]*\]\S*'
            matches = re.findall(pattern, text)
            words_for_table.extend(matches)

            ###### SUJEITOS #######
            sujeito_match = re.search(r'\bSUJEITO:\s*(\S+)', text)
            if sujeito_match:
                sujeito_list.extend([sujeito_match.group(1)] * len(matches))
            else:
                sujeito_list.extend(['Not Specified'] * len(matches))
            
             ##### CURSO #########
            curso_match = re.search(r'\bCURSO:\s*(\S+)', text)
            if curso_match:
                curso_list.extend([curso_match.group(1)] * len(matches))
            else:
                curso_list.extend(['Not Specified'] * len(matches))

             ##### SEXO #######
            sexo_match = re.search(r'\bSEXO:\s*(\S+)', text)
            if sexo_match:
                sexo_list.extend([sexo_match.group(1)] * len(matches))
            else:
                sexo_list.extend(['Not Specified'] * len(matches))

            
             ####### ANO ########
            ano_match = re.search(r'\bANO:\s*(\S+)', text)
            if ano_match:
                ano_list.extend([ano_match.group(1)] * len(matches))
            else:
                ano_list.extend(['Not Specified'] * len(matches))

            ####### CIDADE ########
            cidade_match = re.search(r'\bCIDADE:\s*(\S+)', text)
            if cidade_match:
                cidade_list.extend([cidade_match.group(1)] * len(matches))
            else:
                cidade_list.extend(['Not Specified'] * len(matches))

    df = DataFrame({'palavra': words_for_table, 'sujeito': sujeito_list, 'curso': curso_list, 'sexo': sexo_list, 'ano': ano_list, 'cidade': cidade_list})

    #remove commas and periods at the end of values
    punctuation_pattern = r'[.,…!?]+$'
    a_pattern = r'-a'
    o_pattern = r'-o'
    pattern = punctuation_pattern + '|' + a_pattern + '|' + o_pattern
    df['palavra'] = df['palavra'].str.replace(pattern, '', regex=True)
        
            
    return df

directory_path = "../data/"
df = df_find_cases(directory_path)
df

,palavra,sujeito,curso,sexo,ano,cidade
0,ensina[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
1,se[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
2,ve[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
3,vive[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
4,fala[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA
...,...,...,...,...,...,...
226,Dormi[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
227,fala[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
228,brinca[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA
229,ata[ndʊ][s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA


In [7]:
def create_apagamento(row):
    
    pattern_apagamento = r'\[[^\]]*∅[^\]]*\]'
    

    if re.search(pattern_apagamento, row['palavra']):
        gerundio_apagado = 'Sim'
    else: 
        gerundio_apagado = 'Não'

    return gerundio_apagado

In [8]:
df['apagamento_gerundio']= df.apply(create_apagamento, axis=1)

In [9]:
df

,palavra,sujeito,curso,sexo,ano,cidade,apagamento_gerundio
0,ensina[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,Sim
1,se[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,Sim
2,ve[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,Sim
3,vive[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,Sim
4,fala[n∅dʊ],SUJEITO_6,PEDAGOGIA,MULHER,PRIMEIRO,LONDRINA,Sim
...,...,...,...,...,...,...,...
226,Dormi[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,Sim
227,fala[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,Sim
228,brinca[n∅do],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,Sim
229,ata[ndʊ][s],SUJEITO_1,DIREITO,HOMEM,PRIMEIRO,OUTRA,Não


In [11]:
frequency_table = df['apagamento_gerundio'].value_counts()
percentage_table = df['apagamento_gerundio'].value_counts(normalize=True) * 100

# Combine counts and percentages into a single DataFrame
result_df = pd.DataFrame({'Contagem': frequency_table, 'Porcentagem': percentage_table})
result_df

,Contagem,Porcentagem
apagamento_gerundio,,
Sim,203,87.878788
Não,28,12.121212


In [27]:
cross_table = pd.crosstab(df['sexo'], df['apagamento_gerundio'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_gerundio,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
sexo,,,,
HOMEM,13,107,10.833333,89.166667
MULHER,15,96,13.513514,86.486486


In [28]:
cross_table = pd.crosstab(df['curso'], df['apagamento_gerundio'])
cross_table

apagamento_gerundio,Não,Sim
curso,,
DIREITO,10,64
LETRAS,12,59
PEDAGOGIA,6,80


In [29]:
cross_table = pd.crosstab(df['curso'], df['apagamento_gerundio'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_gerundio,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
curso,,,,
DIREITO,10,64,13.513514,86.486486
LETRAS,12,59,16.901408,83.098592
PEDAGOGIA,6,80,6.976744,93.023256


In [19]:
cross_table = pd.crosstab(df['apagamento_gerundio'], df['curso'])
df_percentages = cross_table.copy()
total = (cross_table['DIREITO'] + cross_table['LETRAS'] + cross_table['PEDAGOGIA'])
df_percentages['DIREITO (%)'] = (cross_table['DIREITO']/total) * 100
df_percentages['LETRAS (%)'] = (cross_table['LETRAS']/total) * 100
df_percentages['PEDAGOGIA (%)'] = (cross_table['PEDAGOGIA']/total) * 100
df_percentages

curso,DIREITO,LETRAS,PEDAGOGIA,DIREITO (%),LETRAS (%),PEDAGOGIA (%)
apagamento_gerundio,,,,,,
Não,10,12,6,35.714286,42.857143,21.428571
Sim,64,59,80,31.527094,29.064039,39.408867


In [30]:
cross_table = pd.crosstab(df['ano'], df['apagamento_gerundio'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages

apagamento_gerundio,Não,Sim,S/ APAGAMENTO (%),C/ APAGAMENTO (%)
ano,,,,
PRIMEIRO,16,107,13.008130,86.991870
ÚLTIMO,12,96,11.111111,88.888889


In [ ]:
cross_table = pd.crosstab(df['curso'], df['apagamento_gerundio'])
df_percentages = cross_table.copy()
total = (cross_table['Sim'] + cross_table['Não'])
df_percentages['S/ APAGAMENTO (%)'] = (cross_table['Não']/total) * 100
df_percentages['C/ APAGAMENTO (%)'] = (cross_table['Sim'] / total) * 100
df_percentages